In [6]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import pandas as pd

#### Conseguir url 

In [7]:
#funcion para para dar forma a url de cada serie
def url_obt(x):
    url_principal="http://www.sensacine.com"
    try :
        y = url_principal+x.find('a').attrs['href']
    except :
        y = 'No hay'
    return y

In [8]:
#almacenando los url de las series(de 321 pag.)
urls=[]
for i in range(1,321):  
    url_tv = "http://www.sensacine.com/series-tv/?page={}".format(i)
    resp = requests.get(url_tv)
    soup = BeautifulSoup(resp.text, 'html.parser')
    lista_series = soup.find_all('li',{'class':'hred'})
    urls.extend(list(map(url_obt,lista_series)))
#eliminar los no hay
urls_array=np.array(urls)#4799
urls_array_t = urls_array[urls_array!='No hay']
urls=urls_array_t.tolist()#4797

#### Scrapear cada pagina

In [9]:
data_list = []
for i in urls:
    resp = requests.get(i)
    soup = BeautifulSoup(resp.text, 'lxml')
    tags_genero=soup.select('.meta-body-item.meta-body-info > span')[2:]
    try :
        sinopsis=soup.find('div',{'class':"content-txt"}).text
    except :
        sinopsis=''
    genero_list = list(map(lambda x : x.text,tags_genero))
    genero=",".join(genero_list)
    fila = [soup.find('div',{'class':"titlebar-title titlebar-title-lg"}).text,
            sinopsis,
            genero]
    data_list.append(fila)

#### Dandole el formato de tabla

In [13]:
tabla_series_tv = pd.DataFrame(data_list,columns=['Titulo','Sinopsis','Genero'])

In [14]:
#limpieza preliminar
tabla_series_tv = tabla_series_tv.assign(Sinopsis = lambda x : x.Sinopsis.replace('\n',' ').str.strip())

In [15]:
tabla_series_tv.head(10)

,Titulo,Sinopsis,Genero
0,#Cartel,#Cartel es una serie dramática de televisión d...,Judicial
1,'Allo 'Allo,La vida es cualquier cosa menos simple para Re...,Comedia
2,(Des)encanto,(Des)encanto es una comedia de animación para ...,"Fantasía,Animación"
3,+ De 100 Mentiras,+ De 100 Mentiras narra la historia de un hack...,Suspense
4,10,,Drama
5,10 Razones para odiarte,"En el colegio Padua High, las hermanas Kat y B...",Comedia
6,10 Years,,Comedia
7,100 Code,Tommy Conley es un detective de homicidios de ...,"Drama,Crimen,Suspense"
8,100 Questions for Charlotte Payne,"Charlotte, una jovenzuela en busca del amor qu...",Comedia
9,1000 maneras de morir,'1000 maneras de morir' es una serie antológic...,"Comedia,Documental"


In [16]:
#guardando en la carpeta datos
tabla_series_tv.to_csv("../datos/tablas_series_tv.csv",sep=";")